In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import openpyxl
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment
from openpyxl.drawing.image import Image
import win32com.client as win32
import os
from PIL import ImageGrab
import win32com.client

olApp = win32.Dispatch('Outlook.Application')
olNS = olApp.GetNameSpace('MAPI')

import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period
from Defs import new_list
from Defs import export_from_RISKCUSTOM
from Defs import add_in_currency_column
from Defs import concat_columns
from Defs import export_from_WHWEEK

In [20]:
Today = '2024-06-30'

print_to_excel = True

In [3]:
query = f"""
select * 
from xxwh."OPRECConso_detailed"
where "periodDate" = TO_DATE('{Today}', 'YYYY-MM-DD')
and "rowType" != 'on hand'
"""
data_OPRECConso_detailed = export_from_WHWEEK(query)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:174: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


In [12]:
data_OPRECConso_detailed_work = data_OPRECConso_detailed[['productGroupNine','businessUnit','customer','appliedPrice','dealPriceDealCurrency','currency','appliedEndDate','quantity','unshippedDealAmount']]
# merge segments
data_OPRECConso_detailed_work['businessSegment'] = merge_SalesUnits(data_OPRECConso_detailed_work,'businessUnit',merge_col='ocpSegment')
data_OPRECConso_detailed_work['customerCode'] = merge_Mapping(data_OPRECConso_detailed_work,'customer')
data_OPRECConso_detailed_work['customerSegment'] = merge_SalesUnits(data_OPRECConso_detailed_work,'customerCode',merge_col='ocpSegment')
# work with data
data_OPRECConso_detailed_work_not_na = data_OPRECConso_detailed_work[data_OPRECConso_detailed_work.unshippedDealAmount!=0].drop('quantity', axis=1).dropna(subset='appliedPrice').reset_index(drop=True)
data_OPRECConso_detailed_work_not_na['Amount_appliedPrice'] = data_OPRECConso_detailed_work_not_na.appliedPrice*data_OPRECConso_detailed_work_not_na.unshippedDealAmount
data_OPRECConso_detailed_work_not_na['Amount_dealPriceDealCurrency'] = data_OPRECConso_detailed_work_not_na.dealPriceDealCurrency*data_OPRECConso_detailed_work_not_na.unshippedDealAmount
data_OPRECConso_detailed_work_not_na = Period(data_OPRECConso_detailed_work_not_na, Today, 'appliedEndDate')



C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['RUFERT' 'RUFERT' 'NAMSALES' ... 'NAMSALES' 'NAMSALES' 'RUFERT']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merge_data.loc[merge_data[f'{id_col}_merge'] != 'External', 'last_merge'] = merge_data.loc[merge_data[f'{id_col}_merge'] != 'External', f'{id_col}_merge']
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_2180\1557821853.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

In [14]:
data_OPRECConso_detailed_work_not_na = add_in_currency_column(data_OPRECConso_detailed_work_not_na, 'USD', 'currency', False, 'Amount_dealPriceDealCurrency', Today)
data_OPRECConso_detailed_work_not_na

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


,productGroupNine,businessUnit,customer,appliedPrice,dealPriceDealCurrency,currency,appliedEndDate,unshippedDealAmount,businessSegment,customerCode,customerSegment,Amount_appliedPrice,Amount_dealPriceDealCurrency,Days,Period,Coef_to_USD,Amount_dealPriceDealCurrency_in_USD
0,MAP,RUBMU,RUEMO,41090.000000,41090.000000,RUB,2024-07-31,-184.000000,RUFERT,RUEMO,RUFERT,-7.560560e+06,-7.560560e+06,31,1M-3M,0.01166,-88153.655832
1,MAP,RUMRD,RUEMO,41090.000000,41090.000000,RUB,2024-07-31,184.000000,RUFERT,RUEMO,RUFERT,7.560560e+06,7.560560e+06,31,1M-3M,0.01166,88153.655832
2,DFP,USBFC,SANDERSON FARMS INC.,680.149576,680.149576,USD,2024-06-30,-20.374665,NAMSALES,External,External,-1.385782e+04,-1.385782e+04,0,<1M,1.00000,-13857.820000
3,NPK MOP,RUNEV,RUEMO,23640.000000,23640.000000,RUB,2024-07-31,-1000.000000,RUFERT,RUEMO,RUFERT,-2.364000e+07,-2.364000e+07,31,1M-3M,0.01166,-275634.665140
4,NPK SOP,CHTRD,DEEAG,463.260000,463.260000,EUR,2024-08-31,-600.000000,TRADING,DEEAG,EURSALES,-2.779560e+05,-2.779560e+05,62,1M-3M,1.07130,-297774.262800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21506,CAN INHIBITED,CHTRD,FREAF,377.190000,377.190000,EUR,2024-09-30,-25.200000,TRADING,FREAF,EURSALES,-9.505188e+03,-9.505188e+03,92,3M-6M,1.07130,-10182.907904
21507,MELAMINE,RUNEV,"OOO ""Ultrakhem""",86500.000000,86500.000000,RUB,2024-07-31,-500.000000,RUFERT,External,External,-4.325000e+07,-4.325000e+07,31,1M-3M,0.01166,-504280.848871
21508,UREA PRILL,USBFC,RIDLEY INC,468.498492,468.498492,USD,2024-07-15,-19.957375,NAMSALES,External,External,-9.350000e+03,-9.350000e+03,15,<1M,1.00000,-9350.000000
21509,UREA PRILL,USBFC,RIDLEY INC,468.498492,468.498492,USD,2024-07-15,-6.350074,NAMSALES,External,External,-2.975000e+03,-2.975000e+03,15,<1M,1.00000,-2975.000000


In [5]:
# mapping_data = data_OPRECConso_detailed_work.loc[data_OPRECConso_detailed_work.customerSegment=='External',['customer','customerCode','customerSegment']].drop_duplicates()
# new_list(mapping_data,Output_file,'mapping_sheet')

In [6]:
query = f"""
select * 
from "RISKACCESS"."unifiedOcp"
where "reportDate" = TO_DATE('{Today}', 'YYYY-MM-DD')
and "sourceTable" in ('flowDealsReportChanges_inventory','unifiedMaterialMovementSlice_inventory')
"""
data = export_from_RISKCUSTOM(query)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


In [7]:
data_work = data[['sourceTable','accountName','holding','buCode','ocpSegment','internalFlag','maturityDateTo','volumeUsd','salesUnitsId']]


In [18]:
intersection_list = set(data_OPRECConso_detailed_work_not_na.productGroupNine.tolist()).intersection(set(data_work.accountName.tolist()))
data_work_query = data_work[data_work.accountName.isin(intersection_list)]
data_unifiedOcp_pivot = data_work_query.pivot_table(index=['holding','buCode','accountName','sourceTable'], values='volumeUsd',aggfunc=sum).reset_index()

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_2180\4107252168.py:3: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  data_unifiedOcp_pivot = data_work_query.pivot_table(index=['holding','buCode','accountName','sourceTable'], values='volumeUsd',aggfunc=sum).reset_index()


In [15]:
data_OPRECConso_query = data_OPRECConso_detailed_work_not_na[(data_OPRECConso_detailed_work_not_na.productGroupNine.isin(intersection_list))]
data_OPRECConso_pivot = data_OPRECConso_query.pivot_table(index=['businessUnit','productGroupNine'], values='Amount_dealPriceDealCurrency_in_USD',aggfunc=sum).reset_index()

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_2180\863434828.py:2: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  data_OPRECConso_pivot = data_OPRECConso_query.pivot_table(index=['businessUnit','productGroupNine'], values='Amount_dealPriceDealCurrency_in_USD',aggfunc=sum).reset_index()


In [32]:
data_USD_merge = data_unifiedOcp_pivot.merge(data_OPRECConso_pivot, how='outer', left_on=['buCode','accountName'], right_on=['businessUnit','productGroupNine'])[['sourceTable','holding','buCode','accountName','volumeUsd','Amount_dealPriceDealCurrency_in_USD']].rename(columns={'volumeUsd':'unifiedOcp','Amount_dealPriceDealCurrency_in_USD':'OPRECConso'}).sort_values(['unifiedOcp','OPRECConso'])
data_USD_merge[['unifiedOcp','OPRECConso']] = data_USD_merge[['unifiedOcp','OPRECConso']].fillna(0)
data_USD_merge['huge_diff'] = ~((data_USD_merge.unifiedOcp.abs()/data_USD_merge.OPRECConso.abs() > 0.7) & (data_USD_merge.unifiedOcp.abs()/data_USD_merge.OPRECConso.abs() < 1.30))
data_USD_merge = data_USD_merge[~((data_USD_merge.unifiedOcp==0)&(data_USD_merge.OPRECConso==0))]
### to excel
if print_to_excel == True:
    Output_file = f'{str(date.today())}_unifiedOcp_vs_OPRECConso.xlsx'
    data_USD_merge.to_excel(Output_file, index=False, sheet_name='diff')
    new_list(data_work, Output_file, sheet_name='unifiedOcp')
    new_list(data_OPRECConso_detailed_work_not_na, Output_file, sheet_name='OPRECConso')    


In [31]:
data_USD_merge[data_USD_merge.huge_diff==False]
# data_USD_merge[(data_USD_merge.unifiedOcp==0)&(data_USD_merge.OPRECConso==0)]

,sourceTable,holding,buCode,accountName,unifiedOcp,OPRECConso,huge_diff
